**Preprocessing of text data in Verbatim column**

In [0]:
#### Take a look at some verbatims
for index,text in enumerate(dsat['Q11._Verbatim'][35:40]):
  print('Verbatim %d:\n'%(index+1),text)



*   **Expand Contractions**




Contractions are the shortened versions of words like don’t for do not and how’ll for how will. These are used to reduce the speaking and writing time of words. We need to expand these contractions for a better analysis of the reviews.

We have used a dictionary of common English contractions that we will use for mapping the contractions to their expanded forms.

In [0]:
# Dictionary of English Contractions
contractions_dict = { "ain't": "are not","'s":" is","aren't": "are not",
                     "can't": "cannot","can't've": "cannot have",
                     "'cause": "because","could've": "could have","couldn't": "could not",
                     "couldn't've": "could not have", "didn't": "did not","doesn't": "does not",
                     "don't": "do not","hadn't": "had not","hadn't've": "had not have",
                     "hasn't": "has not","haven't": "have not","he'd": "he would",
                     "he'd've": "he would have","he'll": "he will", "he'll've": "he will have",
                     "how'd": "how did","how'd'y": "how do you","how'll": "how will",
                     "I'd": "I would", "I'd've": "I would have","I'll": "I will",
                     "I'll've": "I will have","I'm": "I am","I've": "I have", "isn't": "is not",
                     "it'd": "it would","it'd've": "it would have","it'll": "it will",
                     "it'll've": "it will have", "let's": "let us","ma'am": "madam",
                     "mayn't": "may not","might've": "might have","mightn't": "might not", 
                     "mightn't've": "might not have","must've": "must have","mustn't": "must not",
                     "mustn't've": "must not have", "needn't": "need not",
                     "needn't've": "need not have","o'clock": "of the clock","oughtn't": "ought not",
                     "oughtn't've": "ought not have","shan't": "shall not","sha'n't": "shall not",
                     "shan't've": "shall not have","she'd": "she would","she'd've": "she would have",
                     "she'll": "she will", "she'll've": "she will have","should've": "should have",
                     "shouldn't": "should not", "shouldn't've": "should not have","so've": "so have",
                     "that'd": "that would","that'd've": "that would have", "there'd": "there would",
                     "there'd've": "there would have", "they'd": "they would",
                     "they'd've": "they would have","they'll": "they will",
                     "they'll've": "they will have", "they're": "they are","they've": "they have",
                     "to've": "to have","wasn't": "was not","we'd": "we would",
                     "we'd've": "we would have","we'll": "we will","we'll've": "we will have",
                     "we're": "we are","we've": "we have", "weren't": "were not","what'll": "what will",
                     "what'll've": "what will have","what're": "what are", "what've": "what have",
                     "when've": "when have","where'd": "where did", "where've": "where have",
                     "who'll": "who will","who'll've": "who will have","who've": "who have",
                     "why've": "why have","will've": "will have","won't": "will not",
                     "won't've": "will not have", "would've": "would have","wouldn't": "would not",
                     "wouldn't've": "would not have","y'all": "you all", "y'all'd": "you all would",
                     "y'all'd've": "you all would have","y'all're": "you all are",
                     "y'all've": "you all have", "you'd": "you would","you'd've": "you would have",
                     "you'll": "you will","you'll've": "you will have", "you're": "you are",
                     "you've": "you have"}

In [0]:
#### Regular expression for finding contractions
contractions_re=re.compile('(%s)' % '|'.join(contractions_dict.keys()))

In [0]:
#### Function for expanding contractions
def expand_contractions(text,contractions_dict=contractions_dict):
  def replace(match):
    return contractions_dict[match.group(0)]
  return contractions_re.sub(replace, text)

In [0]:
#### Expanding Contractions in the reviews
dsat['verbatim_processed']=dsat['Q11._Verbatim'].apply(lambda x:expand_contractions(x))



*   **Remove unwanted elements**



In [0]:
import re
def preprocessor(text):
  text = re.sub('<[^>]*>', '', text)    #### Remove HTML Elements
  emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)    #### Identify Emoticons
  text = re.sub('[\W]+', ' ', text.lower()) +\
  ' '.join(emoticons).replace('-', '')#### Lowercase Reviews
  text = re.sub('\w*\d\w*','', text)    #### Remove digits and word containing digits
  text = re.sub('[%s]' % re.escape(string.punctuation), '', text)    #### string.punctuations function contains all the punctuations 
  #### and we use regular expressions to search them in the text and remove them
  text = re.sub(' +',' ',text)    #### Remove extra spaces
  return text

In [0]:
#### First Verbatim Original
dsat.loc[0, 'verbatim_processed']

In [0]:
#### First Verbatim after adding processor function
preprocessor(dsat.loc[0, 'verbatim_processed'])

In [0]:
#### Apply preprocessor function to entire dataset
dsat['verbatim_processed'] = dsat['verbatim_processed'].apply(preprocessor)

In [0]:
#### Look how our text appears after cleaning
for index,text in enumerate(dsat['verbatim_processed'][35:40]):
  print('Review %d:\n'%(index+1),text)


  

*   **Remove Stopwords**





In [0]:
#### Remove Stopwords
import spacy
nlp = spacy.load('en_core_web_sm',disable=['parser', 'ner'])

In [0]:
dsat['verbatim_processed'] = dsat['verbatim_processed'].apply(lambda x: ' '.join([token.lemma_ for token in list(nlp(x)) if (token.is_stop==False)]))

In [0]:
#### Look how our text appears after removing stopwords
for index,text in enumerate(dsat['verbatim_processed'][35:40]):
  print('Review %d:\n'%(index+1),text)

=> The other_stopwords can be our dictionary of insignicant words generally present in customer verbatim. Feel Free to add more words to this dictionary if you find it.

In [0]:
#### removal of common words which are not significant to analysis
#### done by visually analyzing the wordcloud below and adding words to other_stopwords
other_stopwords = ['adobe', 'de', 'que', 'und', 'en', 'quot', 'don', 'la', 'chat', 'tell', 'customer', 'representative', 'rep', 'agent', 'da', 'go','able', 'finally', 
                   'case', 'number','acrobat','pro','dc','creative','cloud', 'company']
dsat['verbatim_processed']=dsat['verbatim_processed'].apply(lambda x: " ".join(word for word in x.split() if word not in other_stopwords))

In [0]:
#### Understanding the common words used in customer reviews
all_words = ' '.join([text for text in dsat['verbatim_processed']])
from wordcloud import WordCloud
wordcloud = WordCloud(width=800, height=500, random_state=21, max_font_size=110).generate(all_words)

plt.figure(figsize=(10, 7))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis('off')
plt.show()